In [ ]:
import requests
from extract import unsharp, thresh, gray, scale, blur, process_text, extract_text
import download
import cv2

from datetime import datetime

In [ ]:
url = 'http://192.168.1.17'
runner_id = 'jupyter'

In [ ]:
r = requests.get(url + '/api/videotasks', params={'runner': runner_id})
r

In [ ]:
len(r.content)

In [ ]:
task = r.json()
task

In [ ]:
if task['type'] == 'INFO_REQUEST':
    frames, fps = download.info(task['videoId'])
    response = requests.post(url + '/api/videotasks/info', json={
        'taskId': task['id'],
        'runner': runner_id,
        'frames': frames,
        'fps': fps
    })
    print(response)

In [ ]:
if task['type'] == 'GET_TEXT':
    texts = []
    for i in range(task['frame'], task['frame']+task['count']):
        print('Frame', i)
        image = download.get_frame_num(task['videoId'], i)
        text = process_text(extract_text(unsharp(scale(6, gray(image)))))
        if text:
            print(text)
            texts.append({'frame': i, 'text': text})
    response = requests.post(url + '/api/videotasks/texts', json={
        'taskId': task['id'],
        'runner': runner_id,
        'texts': texts
    })
    print(response)

In [ ]:
target_height = 200

if task['type'] == 'GET_IMAGE':
    frames = task['frames']
    for f in frames:
        image = download.get_frame_num(task['videoId'], f)
        h = image.shape[0]
        w = image.shape[1]
        scale = target_height / h
        im = cv2.resize(image, None, fx=scale, fy=scale)
        _ , encoded_image = cv2.imencode('.png', im)
        requests.post(url+'/api/videotasks/frame/'+task['videoId']+'/'+str(f), data = encoded_image.tobytes())
    requests.post(url+'/api/videotasks/images', json={
        'taskId': task['id'],
        'runner': runner_id,
    })